## Mapping CNV Segments to Genes

### Overview
This notebook uses PyEnsembl's genome metadata to look up genes based on the retrieved CNV data. Chromosome, Start, and End (genomic positions of CNV segment) are used to specify the locus. GRCh38, the most recent and comphrehensive human reference genome assembly, is used via PyEnsembl as well.


PyEnsembl installation and reference genome data (GRCh38) download

In [1]:
!pip install pyensembl
!pyensembl install --release 84 --species homo_sapiens

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.2/537.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 3.5 MB/s eta 0:00:00
  Created wheel for memoized-property: filename=memoized_property-1.0.3-py2.py3-none-any.whl size=4185 sha256=9e2ed916fa853

Import libraries

In [3]:
import pandas as pd
from pyensembl import EnsemblRelease
from google.colab import files

Get CNV data from uploaded files on Google Colab

In [4]:
uploaded = files.upload()
df = pd.read_csv("ground_truth_combined.csv", delimiter=',')
print(df.head())

Saving ground_truth_combined.csv to ground_truth_combined.csv
                            GDC_Aliquot Chromosome  Start        End  \
0  2eb1196d-5e6d-4223-953c-d558c75a8c2e       chr1  13116  248945703   
1  2eb1196d-5e6d-4223-953c-d558c75a8c2e       chr2  10587  242183243   
2  2eb1196d-5e6d-4223-953c-d558c75a8c2e       chr3  18519  198181744   
3  2eb1196d-5e6d-4223-953c-d558c75a8c2e       chr4  11961  190122722   
4  2eb1196d-5e6d-4223-953c-d558c75a8c2e       chr5  11882  181363900   

   Copy_Number  Major_Copy_Number  Minor_Copy_Number    Case_ID  Sample  
0            4                  2                  2  C3L-00359       1  
1            4                  2                  2  C3L-00359       1  
2            4                  2                  2  C3L-00359       1  
3            4                  2                  2  C3L-00359       1  
4            4                  2                  2  C3L-00359       1  


Or retrieve combined CNV data locally

In [ ]:
file_path = "D:/GDC-data/ground_truth_combined.csv"
df = pd.read_csv(file_path, delimiter=',')
print(df.head())

Gene data retrieval function using PyEnsembl

---



In [14]:
# use latest Ensembl release for human genome
data = EnsemblRelease(84)
data.download()
data.index()

# function to get gene data for the locus
def get_gene_data(chromosome, start, end):
    genes = data.genes_at_locus(contig=chromosome, position=start, end=end)
    if not genes:
        return None, None, None, None

    # Choose the longest gene at the locus
    prominent_gene = max(genes, key=lambda gene: gene.end - gene.start)

    gene_name = prominent_gene.name
    gene_biotype = prominent_gene.biotype
    gene_length = prominent_gene.end - prominent_gene.start
    exon_count = len(prominent_gene.exons)

    return gene_name, gene_biotype, gene_length, exon_count

Append gene data columns to existing dataframe

In [19]:
# apply function to each row in dataframe
df[['gene_name', 'gene_biotype', 'gene_length', 'exon_count']] = df.apply(
    lambda row: pd.Series(get_gene_data(row['Chromosome'], row['Start'], row['End'])),
    axis=1
)

print(df.head(10)) # updated dataframe

                            GDC_Aliquot Chromosome     Start        End  \
0  2eb1196d-5e6d-4223-953c-d558c75a8c2e       chr1     13116  248945703   
1  2eb1196d-5e6d-4223-953c-d558c75a8c2e       chr2     10587  242183243   
2  2eb1196d-5e6d-4223-953c-d558c75a8c2e       chr3     18519  198181744   
3  2eb1196d-5e6d-4223-953c-d558c75a8c2e       chr4     11961  190122722   
4  2eb1196d-5e6d-4223-953c-d558c75a8c2e       chr5     11882  181363900   
5  2eb1196d-5e6d-4223-953c-d558c75a8c2e       chr6    100116   32468178   
6  2eb1196d-5e6d-4223-953c-d558c75a8c2e       chr6  32469273   32536402   
7  2eb1196d-5e6d-4223-953c-d558c75a8c2e       chr6  32537412  170740469   
8  2eb1196d-5e6d-4223-953c-d558c75a8c2e       chr7     20608  159334386   
9  2eb1196d-5e6d-4223-953c-d558c75a8c2e       chr8     61774  125589296   

   Copy_Number  Major_Copy_Number  Minor_Copy_Number    Case_ID  Sample  \
0            4                  2                  2  C3L-00359       1   
1            4          

### Gene Data Selection

Specific gene data was chosen to add relevant features to the current ground truth table

- Gene name: for information, common name for interpreting data
- Gene biotype: type of gene, can indicate if the CNV affects protein-coding genes
- Gene length: measurement to check for patterns among CNV segments
- Exon count: checking if the CNV affects specific regions of a gene

### Additional Notes
- PyEnsembl's `gene_at_locus()` returns all genes overlapping at that locus.
- We will map the most "prominent" gene to the corresponding row of the ground truth dataframe.
- Currently the criteria for the prominent gene is the longest gene at the locus (see the lambda function for returning the max gene length).